### Student Information
Name:

Student ID:

GitHub ID:

Kaggle name:

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [1]:
### Begin Assignment Here
lr = 2e-5
batch_size = 64
epochs = 5
grad_clip = 1.0

In [2]:
import pandas as pd
import os

data_path = "dataset/"
df_data = pd.read_json(os.path.join(data_path, "tweets_DM.json"), lines=True)
df_data["_source"][0]

{'tweet': {'hashtags': ['Snapchat'],
  'tweet_id': '0x376b20',
  'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}}

In [3]:
df_data.head()

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [4]:
df_source = pd.json_normalize(df_data["_source"])

In [5]:
df_source.head()

,tweet.hashtags,tweet.tweet_id,tweet.text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [6]:
df_data = pd.concat([df_data, df_source], axis=1)
df_data = df_data.drop(columns=["_source"])

In [7]:
df_data.head()

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [8]:
len(df_data["tweet.hashtags"])

1867535

In [9]:
df_data = df_data.rename(columns={"tweet.hashtags": "hashtags"})
df_data = df_data.rename(columns={"tweet.tweet_id": "tweet_id"})
df_data = df_data.rename(columns={"tweet.text": "text"})
df_data = df_data.rename(columns={"_score": "score"})
df_data = df_data.rename(columns={"_index": "index"})
df_data = df_data.rename(columns={"_crawldate": "crawldate"})
df_data = df_data.rename(columns={"_type": "type"})

In [10]:
df_data.head()

,score,index,crawldate,type,hashtags,tweet_id,text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [11]:
df_identification = pd.read_csv(os.path.join(data_path, "data_identification.csv"))

In [12]:
df_data = pd.merge(df_data, df_identification, on="tweet_id")

In [13]:
df_data.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test


In [14]:
df_test = df_data[df_data["identification"] == "test"]

In [15]:
df_emotion = pd.read_csv(os.path.join(data_path, "emotion.csv"))

In [16]:
df_emotion.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [17]:
df_data = pd.merge(df_data, df_emotion, on="tweet_id")

In [18]:
df_data.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification,emotion
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation


In [19]:
df_train = df_data[df_data["identification"] == "train"]

In [20]:
df_train.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification,emotion
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation


In [21]:
df_test.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test
9,66,hashtag_tweets,2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test
30,104,hashtag_tweets,2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
33,310,hashtag_tweets,2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test


In [22]:
emotion_list = ['None']
for i in range(len(df_train)):
    if df_train["emotion"][i] not in emotion_list:
        emotion_list.append(df_train["emotion"][i])
print(emotion_list)

['None', 'anticipation', 'sadness', 'fear', 'joy', 'anger', 'trust', 'disgust', 'surprise']


In [23]:
emotion_id = {}
id_emotion = {}
for i in range(len(emotion_list)):
    emotion_id[emotion_list[i]] = i
print(emotion_id)

for i in range(len(emotion_list)):
    id_emotion[i] = emotion_list[i]
print(id_emotion)

{'None': 0, 'anticipation': 1, 'sadness': 2, 'fear': 3, 'joy': 4, 'anger': 5, 'trust': 6, 'disgust': 7, 'surprise': 8}
{0: 'None', 1: 'anticipation', 2: 'sadness', 3: 'fear', 4: 'joy', 5: 'anger', 6: 'trust', 7: 'disgust', 8: 'surprise'}


In [24]:
df_train["emotion_id"] = df_train["emotion"].apply(lambda x: emotion_id[x])
df_train.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification,emotion,emotion_id
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,1
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,2
2,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,3
3,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,4
4,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,1


In [25]:
df_test["emotion"] = "None"
df_test["emotion_id"] = df_test["emotion"].apply(lambda x: emotion_id[x])
df_test.head()

,score,index,crawldate,type,hashtags,tweet_id,text,identification,emotion,emotion_id
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,None,0
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,None,0
9,66,hashtag_tweets,2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test,None,0
30,104,hashtag_tweets,2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,None,0
33,310,hashtag_tweets,2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test,None,0


In [26]:
import random

random.seed(42)

df_val = df_train.sample(frac=0.2)
df_train = df_train.drop(df_val.index)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion-multilabel-latest")

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
    def __init__(self, dataframe):
        # Use the provided dataframe
        self.data = dataframe[['text', 'emotion_id']]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if isinstance(idx, int):
            # Tokenize the text
            tokenized_text = tokenizer(self.data.iloc[idx]['text'], padding='max_length', truncation=True, return_tensors='pt')
            sample = {
                'input_ids': tokenized_text['input_ids'].squeeze(0),
                'attention_mask': tokenized_text['attention_mask'].squeeze(0),
                'emotion_id': torch.tensor(self.data.iloc[idx]['emotion_id'])
            }
            return sample
        else:
            raise TypeError("Index must be an integer")

In [29]:
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    emotion_ids = [item['emotion_id'] for item in batch]

    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(attention_mask),
        'emotion_id': torch.tensor(emotion_ids)
    }

In [30]:
train_dataset = EmotionDataset(df_train)
dl_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
validation_dataset = EmotionDataset(df_val)
dl_validation = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataset = EmotionDataset(df_test)
dl_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [31]:
for batch in dl_train:
    print(batch)
    break

{'input_ids': tensor([[    0,  2847,  1372,  ...,     1,     1,     1],
        [    0,  1039, 20158,  ...,     1,     1,     1],
        [    0,  2527,  5074,  ...,     1,     1,     1],
        ...,
        [    0, 25800,    28,  ...,     1,     1,     1],
        [    0,  1711,  7009,  ...,     1,     1,     1],
        [    0, 27000, 15899,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'emotion_id': tensor([4, 2, 2, 2, 2, 1, 4, 4, 4, 4, 7, 4, 4, 8, 8, 4, 4, 7, 6, 4, 7, 4, 4, 4,
        4, 4, 4, 1, 4, 3, 6, 4, 4, 6, 2, 1, 4, 4, 3, 7, 4, 6, 3, 6, 1, 4, 6, 4,
        6, 6, 6, 4, 7, 4, 3, 2, 3, 4, 6, 1, 4, 2, 7, 7])}


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion-multilabel-latest")
model = model.to(device)

cuda


2024-11-14 22:43:21.708109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731595401.728215   75181 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731595401.735450   75181 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 22:43:21.763331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

optimizer = optim.AdamW(model.parameters(), lr=lr)
loss_fn = CrossEntropyLoss()

In [34]:
from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss = 0
    pbar = tqdm(dl_train, desc=f"Epoch {epoch + 1}")
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        emotion_id = batch['emotion_id'].to(device)
    
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, emotion_id)
        total_loss += loss.item()

        
        # Calculate accuracy
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        pbar.set_postfix({'loss': loss.item()})
    
    print(f"Training loss: {total_loss / len(dl_train)}")

    model.eval()
    total_loss = 0
    total_correct = 0
    phar = tqdm(dl_validation, desc=f"Epoch {epoch + 1}")
    with torch.no_grad():
        for batch in phar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            emotion_id = batch['emotion_id'].to(device)
        
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, emotion_id)
            total_loss += loss.item()
        
            # Calculate accuracy
            predictions = torch.argmax(logits, dim=-1)
            total_correct += torch.sum(predictions == emotion_id).item()
        
        print(f"Validation loss: {total_loss / len(dl_validation)}")
        print(f"Validation accuracy: {total_correct / len(df_val)}")

Epoch 1: 100%|██████████| 18195/18195 [2:59:42<00:00,  1.69it/s, loss=0.976]  


Training loss: 1.0022500642039154


Epoch 1: 100%|██████████| 4549/4549 [15:57<00:00,  4.75it/s]


Validation loss: 0.9124444525932107
Validation accuracy: 0.6731234950002233


Epoch 2: 100%|██████████| 18195/18195 [2:59:51<00:00,  1.69it/s, loss=0.853]  


...

Validation loss: 0.8973141868092459
Validation accuracy: 0.6876951561764676


Epoch 5: 100%|██████████| 18195/18195 [2:59:29<00:00,  1.69it/s, loss=0.527]  


Training loss: 0.6553126371457177


Epoch 5: 100%|██████████| 4549/4549 [15:55<00:00,  4.76it/s]

Validation loss: 0.960036971184363
Validation accuracy: 0.6880627110434779
